In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn import metrics
import joblib                                                                                                                                            
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import auc
from sklearn.preprocessing import MinMaxScaler

from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from scipy.stats import pearsonr



In [6]:
fajiao = pd.read_table("../data-2/fajiao-condition-4.11.txt",sep='\t',index_col=0)
fajiao = pd.get_dummies(fajiao, columns=fajiao.columns) 
fajiao.to_csv("./result/ont_hot.csv",sep=',')

In [7]:

fajiao = pd.read_table("../data-2/fajiao-condition-4.11.txt",sep='\t',index_col=0)
fajiao = pd.get_dummies(fajiao, columns=fajiao.columns)

fajiao['label'] = fajiao.groupby(fajiao.columns.tolist()).ngroup()


fajiao_drop = fajiao.drop_duplicates()
fajiao_drop = fajiao_drop.drop('label', axis=1)

X = np.array(fajiao_drop)
# X

# X  = StandardScaler().fit_transform(X)
X  = Normalizer().fit_transform(X)
# # X  = MinMaxScaler().fit_transform(X)
# X  = StandardScaler().fit_transform(X)
X.shape

(13, 17)

In [3]:
data = pd.read_table('../data-2/total4.15.txt',sep='\t',index_col=0)
data_fjg= data.loc[:, ~data.columns.str.contains('BC|NT|MC|PC')].T
data_fjg.columns = data_fjg.iloc[0]
data_fjg = data_fjg[1:]
data_fjg = data_fjg.sort_index()
data_fjg = data_fjg.replace("#DIV/0!", 0)
data_fjg = data_fjg.apply(pd.to_numeric)
data_fjg.fillna(0, inplace=True) 
# data_fjg

data_fjg.index = data_fjg.index.astype(float) 
data_fjg = data_fjg.sort_index() 
selected_columns = ['17431-03-7', '505-10-2', '74685-45-3', '622-78-6', '60-12-8', '104-46-1', '98-85-1']
data_fjg = data_fjg.loc[:, selected_columns]
# data_fjg

data_fjg['label'] = fajiao['label']
fajiao_mean = data_fjg.groupby('label').mean()

fajiao_mean_unique = fajiao_mean.reset_index().drop_duplicates()
fajiao_mean_unique = fajiao_mean_unique.drop('label', axis=1)
fajiao_mean_unique

Y = np.array(fajiao_mean_unique)
# 
Y  = Normalizer().fit_transform(Y)
# Y  = MinMaxScaler().fit_transform(Y)
# Y  = StandardScaler().fit_transform(Y)
Y

array([[0.01091579, 0.00528317, 0.37246925, 0.01534858, 0.75271315,
        0.        , 0.54250063],
       [0.01268837, 0.00460412, 0.        , 0.01190042, 0.99983808,
        0.        , 0.        ],
       [0.00282463, 0.00190912, 0.15753617, 0.00947481, 0.98746188,
        0.        , 0.        ],
       [0.00278109, 0.00132184, 0.18105391, 0.01154121, 0.98340063,
        0.        , 0.        ],
       [0.00619448, 0.00514156, 0.12832809, 0.01539357, 0.86177787,
        0.        , 0.49047838],
       [0.01000131, 0.00134881, 0.08268072, 0.0082045 , 0.99649121,
        0.        , 0.        ],
       [0.00917529, 0.00490662, 0.1812818 , 0.01548303, 0.98325425,
        0.        , 0.        ],
       [0.01578562, 0.00158831, 0.10958528, 0.01211092, 0.99377698,
        0.        , 0.        ],
       [0.00999906, 0.001008  , 0.39656576, 0.01610951, 0.89460715,
        0.        , 0.20506861],
       [0.0129659 , 0.00262197, 0.54595782, 0.01076574, 0.82880949,
        0.        , 0.1

In [4]:
fajiao_mean_unique


ID,17431-03-7,505-10-2,74685-45-3,622-78-6,60-12-8,104-46-1,98-85-1
0,821443.322,397572.664000,2.802933e+07,1.155023e+06,5.664372e+07,0.0,40824655.0
1,1390514.500,504564.000000,0.000000e+00,1.304163e+06,1.095719e+08,0.0,0.0
2,342916.000,231770.630000,1.912521e+07,1.150261e+06,1.198799e+08,0.0,0.0
3,311788.000,148192.000000,2.029798e+07,1.293886e+06,1.102492e+08,0.0,0.0
4,457241.664,379520.600000,9.472455e+06,1.136266e+06,6.361158e+07,0.0,36204347.2
5,1259983.500,169925.500000,1.041627e+07,1.033618e+06,1.255398e+08,0.0,0.0
6,1086539.400,581042.290000,2.146742e+07,1.833502e+06,1.164371e+08,0.0,0.0
7,1857549.500,186902.000000,1.289529e+07,1.425135e+06,1.169413e+08,0.0,0.0
8,835707.885,84247.250000,3.314444e+07,1.346411e+06,7.477008e+07,0.0,17139362.0
9,1127405.000,227984.400000,4.747188e+07,9.360979e+05,7.206627e+07,0.0,10547336.8


In [5]:

from sklearn.svm import SVR
base_model = SVR(C=0.1)

In [6]:
gbr_model = MultiOutputRegressor(base_model)

cv = KFold(n_splits=5, shuffle=True, random_state=42)

MSE = []
RMSE = []
MAE = []
PCC = []
R2 = []
adjusted_r2 = []

MSE_train = []
RMSE_train = []
MAE_train = []
PCC_train = []
R2_train = []
adjusted_r2_train = []


for train_idx, test_idx in cv.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, y_test = Y[train_idx], Y[test_idx]
    gbr_model.fit(X_train, Y_train)
    # print(X_train.shape, Y_train.shape)
    #train
    x_pred = gbr_model.predict(X_train)
    mse = mean_squared_error(Y_train, x_pred)
    rmse = np.sqrt(mean_squared_error(Y_train, x_pred))
    mae = np.mean(np.abs(Y_train - x_pred))
    correlation_array = np.empty((len(Y_train), 1))
    for i in range(len(Y_train)):
        correlation = np.corrcoef(Y_train[i], x_pred[i])[0, 1]
        correlation_array[i] = correlation
    pcc = np.mean(correlation_array)
    r2 = r2_score(Y_train, x_pred) 
    num_features = 6
    adj_r2 = 1 - (1 - r2) * (len(Y_train) - 1) / (len(Y_train) - num_features - 1)
    MSE_train .append(mse)
    RMSE_train .append(rmse)
    MAE_train .append(mae)
    PCC_train .append(pcc)
    R2_train .append(r2)
    adjusted_r2_train .append(adj_r2)

    #test
    y_pred = gbr_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = np.mean(np.abs(y_test - y_pred))
    correlation_array = np.empty((len(y_test), 1))
    for i in range(len(y_test)):
        correlation = np.corrcoef(y_test[i], y_pred[i])[0, 1]
        correlation_array[i] = correlation
    pcc = np.mean(correlation_array)
    r2 = r2_score(y_test, y_pred) 
    num_features = 6
    adj_r2 = 1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - num_features - 1)
    MSE.append(mse)
    RMSE.append(rmse)
    MAE.append(mae)
    PCC.append(pcc)
    R2.append(r2)
    adjusted_r2.append(adj_r2)


for i, (a1,a2,a3,a4,a5,a6) in enumerate(zip(MSE_train, RMSE_train,MAE_train, PCC_train,R2_train,adjusted_r2_train), 1):
    print(f'Fold {i}: MSE = {a1:.4f}, RMSE = {a2:.4f},MAE = {a3:.4f}, PCC = {a4:.4f}, R2 = {a5:.4f}, adjusted_r2 = {a6:.4f}')


average_MSE= sum(MSE_train) / len(MSE_train)
average_RMSE = sum(RMSE_train) / len(RMSE_train)
average_MAE= sum(MAE_train) / len(MAE_train)
average_PCC = sum(PCC_train) / len(PCC_train)
average_R2 = sum(R2_train) / len(R2_train)
average_adjusted_r2 = sum(adjusted_r2_train) / len(adjusted_r2_train)

print(f'Average MSE_train = {average_MSE:.7f}')
print(f'Average RMSE_train = {average_RMSE:.7f}')
print(f'Average MAE_train = {average_MAE:.7f}')
print(f'Average PCC_train = {average_PCC:.7f}')
# print(f'Average R2_train = {average_R2:.7f}')
# print(f'Average adjusted_R2_train = {average_adjusted_r2:.7f}')


for i, (a1,a2,a3,a4,a5,a6) in enumerate(zip(MSE, RMSE,MAE, PCC,R2,adjusted_r2), 1):
    print(f'Fold {i}: MSE = {a1:.4f}, RMSE = {a2:.4f},MAE = {a3:.4f}, PCC = {a4:.4f}, R2 = {a5:.4f}, adjusted_r2 = {a6:.4f}')


average_MSE= sum(MSE) / len(MSE)
average_RMSE = sum(RMSE) / len(RMSE)
average_MAE= sum(MAE) / len(MAE)
average_PCC = sum(PCC) / len(PCC)
average_R2 = sum(R2) / len(R2)
average_adjusted_r2 = sum(adjusted_r2) / len(adjusted_r2)

print(f'Average MSE = {average_MSE:.7f}')
print(f'Average RMSE = {average_RMSE:.7f}')
print(f'Average MAE = {average_MAE:.7f}')
print(f'Average PCC = {average_PCC:.7f}')
# print(f'Average R2 = {average_R2:.7f}')
# print(f'Average adjusted_R2 = {average_adjusted_r2:.7f}')

Fold 1: MSE = 0.0045, RMSE = 0.0669,MAE = 0.0367, PCC = 0.9821, R2 = 0.1337, adjusted_r2 = -1.5989
Fold 2: MSE = 0.0075, RMSE = 0.0865,MAE = 0.0492, PCC = 0.9678, R2 = 0.1485, adjusted_r2 = -1.5546
Fold 3: MSE = 0.0062, RMSE = 0.0786,MAE = 0.0458, PCC = 0.9745, R2 = 0.0910, adjusted_r2 = -1.7271
Fold 4: MSE = 0.0067, RMSE = 0.0821,MAE = 0.0462, PCC = 0.9719, R2 = 0.1524, adjusted_r2 = -1.1191
Fold 5: MSE = 0.0063, RMSE = 0.0792,MAE = 0.0463, PCC = 0.9740, R2 = 0.1768, adjusted_r2 = -1.0579
Average MSE_train = 0.0062329
Average RMSE_train = 0.0786776
Average MAE_train = 0.0448417
Average PCC_train = 0.9740391
Fold 1: MSE = 0.0181, RMSE = 0.1345,MAE = 0.0704, PCC = 0.9148, R2 = -1.1841, adjusted_r2 = 2.0921
Fold 2: MSE = 0.0054, RMSE = 0.0736,MAE = 0.0451, PCC = 0.9821, R2 = -2.4164, adjusted_r2 = 2.7082
Fold 3: MSE = 0.0137, RMSE = 0.1169,MAE = 0.0664, PCC = 0.9411, R2 = -1.7764, adjusted_r2 = 2.3882
Fold 4: MSE = 0.0058, RMSE = 0.0762,MAE = 0.0489, PCC = 0.9787, R2 = -1706.8230, adjust

In [7]:

yp = gbr_model.predict(X)
# np.savetxt('my_array.txt', yp, delimiter='\t')
yp.shape  

(13, 7)

In [10]:
yp
result = pd.DataFrame(yp,index = fajiao_drop.index,columns=selected_columns)
result

,17431-03-7,505-10-2,74685-45-3,622-78-6,60-12-8,104-46-1,98-85-1
4.2,0.792656,0.792656,0.764064,0.792656,0.000000,0.0,1.000000
5.2,1.000000,1.000000,0.000000,1.000000,1.000000,0.0,0.056229
6.2,0.792360,0.792360,0.355603,0.792360,0.802586,0.0,0.049581
16.1,0.812847,0.812847,0.585392,0.812847,0.301767,0.0,0.774743
17.1,0.739241,0.739241,0.461672,0.739241,0.334970,0.0,0.862719
23.1,0.857388,0.857388,0.264256,0.857388,0.859703,0.0,0.048136
24.1,0.853472,0.853472,0.322817,0.853472,0.798566,0.0,0.127148
25.1,0.870321,0.870321,0.341353,0.870321,0.810126,0.0,0.054630
26.1,0.126411,0.126411,0.897649,0.126411,0.374795,0.0,0.084519
27.1,0.000000,0.000000,1.000000,0.000000,0.286125,0.0,0.065834


In [9]:

yp  = Normalizer().fit_transform(yp)
yp = MinMaxScaler().fit_transform(yp)
print(yp.shape)
loaded_model = joblib.load("/data/public/dulin/fwwz/code-2/result/SVM_Classifier-7f.joblib")
y_pred = loaded_model.predict(yp).tolist() 
y_pred 

(13, 7)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [10]:

df13 = fajiao.drop_duplicates()

df54 = pd.read_csv("fjg_label.csv",index_col=0)
df54['code'] = fajiao['label']
df54['mean'] = df54.groupby('code')['label'].transform('mean')
df54['mean_label'] = np.where(df54['mean'] > 0.6, 1, 0)
df54

df54 = df54[df54.index.isin(df13.index)]
truth_simi = df54['mean_label'].tolist()
truth_simi

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [11]:
print('precision（test）:',metrics.precision_score(truth_simi,y_pred))
print('recall（test）:',metrics.recall_score(truth_simi,y_pred))
print('Accuracy（test）:',metrics.accuracy_score(truth_simi,y_pred))
# print('F1 （test）:',f1_score(truth_simi,y_pred))
print("y_test:",truth_simi,"\ny_pred:",y_pred)
fpr, tpr, thresholds = roc_curve(truth_simi, y_pred)
auc_1 = auc(fpr, tpr)
print("AUC : ", auc_1)

precision（测试集）: 1.0
recall（测试集）: 1.0
Accuracy（测试集）: 1.0
y_test: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0] 
y_pred: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
AUC :  1.0


In [12]:
yp

array([[0.7926556 , 0.7926556 , 0.76406431, 0.7926556 , 0.        ,
        0.        , 1.        ],
       [1.        , 1.        , 0.        , 1.        , 1.        ,
        0.        , 0.05622887],
       [0.79235986, 0.79235986, 0.3556032 , 0.79235986, 0.80258637,
        0.        , 0.04958142],
       [0.81284659, 0.81284659, 0.58539192, 0.81284659, 0.30176686,
        0.        , 0.77474322],
       [0.73924091, 0.73924091, 0.46167236, 0.73924091, 0.33496995,
        0.        , 0.86271928],
       [0.85738816, 0.85738816, 0.26425573, 0.85738816, 0.85970296,
        0.        , 0.04813564],
       [0.85347182, 0.85347182, 0.32281699, 0.85347182, 0.7985665 ,
        0.        , 0.12714806],
       [0.87032082, 0.87032082, 0.34135348, 0.87032082, 0.81012609,
        0.        , 0.05462955],
       [0.12641146, 0.12641146, 0.89764929, 0.12641146, 0.37479476,
        0.        , 0.08451913],
       [0.        , 0.        , 1.        , 0.        , 0.28612512,
        0.        , 0.0